In [4]:
import tensorflow as tf

import numpy as np

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Input, Lambda, GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image

print("Num GPUs Available: ", tf.config.list_physical_devices('GPU'))

Num GPUs Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [7]:
batch_size = 32
epochs = 5
filter_size = 3
group_1_dim = 32
group_2_dim = 64
group_3_dim = 128
dense_dim = 1024

dropout_rate = 0.2

In [8]:
train_ds = tf.keras.utils.image_dataset_from_directory(r"C:\Users\kyley\Desktop\CS\C147\C147 Amazon Classification\img_uniform_split\train", 
                                                 labels='inferred', 
                                                 image_size=(64,64), 
                                                 batch_size=batch_size,
                                                 validation_split=0.2,
                                                 subset="training",
                                                 seed=123)

val_ds = tf.keras.utils.image_dataset_from_directory(r"C:\Users\kyley\Desktop\CS\C147\C147 Amazon Classification\img_uniform_split\train", 
                                                 labels='inferred', 
                                                 image_size=(64,64), 
                                                 batch_size=batch_size,
                                                 validation_split=0.2,
                                                 subset="validation",
                                                 seed=123)

test_ds = tf.keras.utils.image_dataset_from_directory(r"C:\Users\kyley\Desktop\CS\C147\C147 Amazon Classification\img_uniform_split\test", 
                                                 labels='inferred', 
                                                 image_size=(64,64), 
                                                 batch_size=batch_size)

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

Found 89663 files belonging to 10 classes.
Using 71731 files for training.


KeyboardInterrupt: 

In [3]:
def preprocess_data(X, Y):
  """
  Method:
      pre-processes the data for your model.
  """
  X_p = tf.keras.applications.inception_v3.preprocess_input(X)
  Y_p = to_categorical(Y, 10)

  return X_p, Y_p

X_train_p, Y_train_p = preprocess_data(x_train, y_train)
X_test_p, Y_test_p = preprocess_data(x_test, y_test)

In [4]:
base_inception = InceptionV3(include_top=False,
                             input_shape=(299, 299, 3))

input_layer = Input(shape=(32, 32, 3))

resizing_layer = Lambda(lambda image:                     \
                tf.keras.preprocessing.image.smart_resize(image, \
                (299, 299)))(input_layer)

inception_layers = base_inception(resizing_layer, training=False)

glob_pooling = GlobalAveragePooling2D()(inception_layers)
layer_i = Dense(500, activation='relu')(glob_pooling)
dropout_layer = Dropout(0.3)(layer_i)
output_layer =  Dense(10, activation='softmax')(dropout_layer)

model = Model(inputs=input_layer, outputs=output_layer)

In [5]:
# Freeze the base network
base_inception.trainable = False

model.compile(tf.keras.optimizers.Adam(learning_rate=.0001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

history = model.fit(
        X_train_p,
        Y_train_p,
        validation_data=(X_test_p, Y_test_p),
        batch_size=300,
        epochs=4,
        verbose=1)

results = model.evaluate(X_test_p, Y_test_p)

Epoch 1/4


167/167 [==============================] - 1443s 9s/step - loss: 0.8232 - accuracy: 0.7413 - val_loss: 0.4805 - val_accuracy: 0.8430
Epoch 2/4
167/167 [==============================] - 1258s 8s/step - loss: 0.4751 - accuracy: 0.8419 - val_loss: 0.4312 - val_accuracy: 0.8578
Epoch 3/4
167/167 [==============================] - 1139s 7s/step - loss: 0.4232 - accuracy: 0.8585 - val_loss: 0.4062 - val_accuracy: 0.8637
Epoch 4/4
313/313 [==============================] - 191s 611ms/step - loss: 0.3903 - accuracy: 0.8700
